In [1]:
from functions import BECHMARK_FUNCTIONS, FUNCTION_RANGES
import numpy as np
import time
import optuna
from optuna.samplers import RandomSampler, TPESampler
from tqdm.auto import tqdm
import pandas as pd
from ray import tune
from ray.tune.search import BasicVariantGenerator
from ray.tune.search.hyperopt import HyperOptSearch
from hyperopt import fmin, tpe, rand, hp, Trials
from itertools import product
from metagen.framework import Domain
from metagen.metaheuristics import RandomSearch, TPE
import random
optuna.logging.set_verbosity(optuna.logging.WARNING)

C:\Users\manjj\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
ITERATIONS = 1000
SEED = 0

def optuna_objective(function_name: str, trial):
    x = np.array([trial.suggest_uniform(f'x{i}', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1]) for i in range(2)])
    return BECHMARK_FUNCTIONS[function_name](x)

def run_optuna(function_name: str, search_algorithm: str="random"):

    random.seed(SEED)
    np.random.seed(SEED)
    start_time = time.time()
    study = optuna.create_study(direction='minimize', sampler= TPESampler(seed=SEED) if search_algorithm=="tpe" else RandomSampler(seed=SEED))
    study.optimize(lambda trial: optuna_objective(function_name, trial), n_trials=ITERATIONS, show_progress_bar=False)
    end_time = time.time()

    return {"library": "optuna", "search_algorithm": search_algorithm, "function_name": function_name, "best_value": study.best_value, "best_x0": study.best_params["x0"], "best_x1": study.best_params["x1"], "time": end_time - start_time}

def raytune_objective(config, function_name: str):
    x = np.array([config[f'x{i}'] for i in range(2)])

    return {"score": BECHMARK_FUNCTIONS[function_name](x)}

def run_raytune(function_name: str, search_algorithm: str="random"):

    random.seed(SEED)
    np.random.seed(SEED)
    tune.seed(42)
    search_space = {f'x{i}': tune.uniform(FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1]) for i in range(2)}
    
    start_time = time.time()
    analysis = tune.run(
        tune.with_parameters(raytune_objective, function_name=function_name),
        config=search_space,
        num_samples=ITERATIONS,
        search_alg=HyperOptSearch(random_state_seed=SEED) if search_algorithm=="tpe" else BasicVariantGenerator(random_state=SEED),
        verbose=0,
        resources_per_trial={"cpu": 1}
    )
    end_time = time.time()
    
    best_trial = analysis.get_best_trial(metric="score", mode="min")
    best_config = best_trial.config
    best_loss = best_trial.metric_analysis["score"]["min"]
    
    return {"library": "raytune", "search_algorithm": search_algorithm, "function_name": function_name, "best_value": best_loss, "best_x0": best_config["x0"], "best_x1": best_config["x1"], "time": end_time - start_time}

def hyperopt_objective(params, function_name: str):
    x = np.array([params[f'x{i}'] for i in range(2)])
    return BECHMARK_FUNCTIONS[function_name](x)

def run_hyperopt(function_name: str, search_algorithm: str="random"):
    search_space = {f'x{i}': hp.uniform(f'x{i}', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1]) for i in range(2)}

    random.seed(SEED)
    np.random.seed(SEED)
    
    trials = Trials()
    start_time = time.time()
    best = fmin(
        fn=lambda params: hyperopt_objective(params, function_name),
        space=search_space,
        algo=tpe.suggest if search_algorithm == "tpe" else rand.suggest,
        max_evals=ITERATIONS,
        trials=trials,
        show_progressbar=False,
        rstate=np.random.default_rng(SEED)
    )
    end_time = time.time()
    
    best_loss = min(trials.losses())
    
    return {"library": "hyperopt", "search_algorithm": search_algorithm, "function_name": function_name, "best_value": best_loss, "best_x0": best['x0'], "best_x1": best['x1'], "time": end_time - start_time}

def metagen_objective(solution, function_name: str):
    x = np.array([solution[f'x{i}'] for i in range(2)])
    return BECHMARK_FUNCTIONS[function_name](x)

def run_metagen(function_name: str, search_algorithm: str="random"):

    random.seed(SEED)
    np.random.seed(SEED)

    domain = Domain()
    
    domain.define_real(f'x0', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1])
    domain.define_real(f'x1', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1])

    
    if search_algorithm == "tpe":
        print(search_algorithm)

        search = TPE(domain, lambda solution: metagen_objective(solution, function_name), population_size=10, max_iterations=ITERATIONS//10)
    else:
        search = RandomSearch(domain, lambda solution: metagen_objective(solution, function_name), population_size=ITERATIONS-ITERATIONS//10, max_iterations=ITERATIONS//10)
    
    start_time = time.time()
    best_solution = search.run()
    end_time = time.time()

    return {"library": "metagen", "search_algorithm": search_algorithm, "function_name": function_name, "best_value": best_solution.get_fitness(), "best_x0": best_solution["x0"], "best_x1": best_solution["x1"], "time": end_time - start_time}

In [3]:

results = []
for function_name, search_algorithm in tqdm(list(product(list(BECHMARK_FUNCTIONS.keys())[:1], ["random", "tpe"]))):
    solution_optuna = run_optuna(function_name, search_algorithm)
    solution_hyperopt = run_hyperopt(function_name, search_algorithm)
    solution_metagen = run_metagen(function_name, search_algorithm)
     #solution_raytune = run_raytune(function_name, search_algorithm)
    results.extend([solution_optuna, solution_hyperopt, solution_metagen]) #, solution_raytune])

#.to_csv("optuna_results.csv", index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\manjj\AppData\Local\Temp\ipykernel_27276\3769260791.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  x = np.array([trial.suggest_uniform(f'x{i}', FUNCTION_RANGES[function_name][0], FUNCTION_RANGES[function_name][1]) for i in range(2)])


tpe


E:\Proyectos\MetaGen\src\metagen\metaheuristics\tpe\tpe.py:93: RuntimeWarning: invalid value encountered in scalar divide
  if p_best / (p_best + p_worst) > np.random.rand():


In [4]:
total_results = pd.DataFrame(results)
total_results.groupby([ "function_name", "library", "search_algorithm"]).first()#.to_excel("results.xlsx")

best_value   best_x0   best_x1  \
function_name library  search_algorithm                                   
sphere        hyperopt random              0.018004 -0.075121  0.111178   
                       tpe                 0.001524 -0.038599 -0.005854   
              metagen  random              0.000015  0.002646 -0.002772   
                       tpe                 0.000027 -0.004445  0.002671   
              optuna   random              0.093254 -0.204177  0.227081   
                       tpe                 0.000194  0.010674  0.008925   

                                             time  
function_name library  search_algorithm            
sphere        hyperopt random            0.822616  
                       tpe               3.865770  
              metagen  random            1.167021  
                       tpe               0.431001  
              optuna   random            0.245595  
                       tpe               7.569082

In [5]:
total_results = pd.DataFrame(results)
total_results.groupby([ "function_name", "library", "search_algorithm"]).first()#.to_excel("results.xlsx")

best_value   best_x0   best_x1  \
function_name library  search_algorithm                                   
sphere        hyperopt random              0.018004 -0.075121  0.111178   
                       tpe                 0.001524 -0.038599 -0.005854   
              metagen  random              0.000015  0.002646 -0.002772   
                       tpe                 0.000027 -0.004445  0.002671   
              optuna   random              0.093254 -0.204177  0.227081   
                       tpe                 0.000194  0.010674  0.008925   

                                             time  
function_name library  search_algorithm            
sphere        hyperopt random            0.833652  
                       tpe               3.779283  
              metagen  random            1.468352  
                       tpe               0.432921  
              optuna   random            0.250328  
                       tpe               7.559689